In [25]:
# 0. preamble
from pathlib import Path

%reload_ext autoreload
%autoreload 2

from pyfiles.scenario_functions import load_energyplan_file, format_value, build_params

# Create Scenario file

This notebook takes the `IDA2045_nuclear_flexible.txt` and changes only the inputs, where our analysis differs. The idea behind this is clarity and overview:

## 1. Cases

In [26]:
# parameters
year = '2023-2025'  # for VP
case = 'shock'      # case specifics
pp2  = 5400         # internalization of unstability, run specific (shock: 5400, base: 5800)
out_path = rf'..\ZipEnergyPLAN163\energyPlan Data\Data\{year}_{case}.txt'

### 1.1. Specify changes

In [27]:
# 1. Changes
base_params = {

    # variables --------------- # value ------- # description --------------------- # source -------------------
 
    # demand
    'Input_el_demand_Twh'       : 311/10,       # electricity demand                # KF25
    'Input_add_el_TWh'          : 156/10,       # additional electricity demand     # KF25 (datacentre og ...)

    # production
    'input_RES1_capacity'       : 5150,         # on-shore wind                     # KF25
    'input_RES3_capacity'       : 25134,        # solar PV                          # KF25
    'input_cap_pp2_el'          : pp2,          # Extra power plant cap             # FOA (1700) added 4.3GW
    
    # variation patterns
    'Filnavn_elbehov'           : f'{year}_Electricity_Demand.txt',                 # ENS
    'Filnavn_wind'              : f'{year}_offshore_prod.txt',                      # ENS
    'Filnavn_wave'              : f'{year}_onshore_prod.txt',                       # ENS
    'Filnavn_pv'                : f'{year}_solar_prod.txt',                         # ENS
    'Filnavn_prices'            : f'{year}_ENS_elspotprices.txt',                   # ENS
    'filnavn_nuclear'           : 'el_atomkraft_dh.txt',  
    'Filnavn_cshp'              : 'varme_atomkraft_dh.txt',  # alternative is nuclear=const.txt 

    # costs
    'input_Inv_Nuclear'         : 43/10,        # investment cost nuclear           # OECD IEA
    'input_Period_Nuclear'      : 60,           # economic lifetime nuclear         # FOA
    'input_FOM_Nuclear'         : 52/10,        # fixed O&M cost nuclear            # OECD IEA
    'input_Inv_WindOffshore'    : 25/10,        # investment cost offshore wind     # FL
    'input_Period_WindOffshore' : 30,           # economic lifetime offshore wind   # FOA
    'input_FOM_WindOffshore'    : 19/10,        # fixed O&M cost offshore wind      # FOA
}

# 2. base scenario
base_case_params = {
    'input_nuclear_cap'         : 0,            # base nuclear capacity             # FL + LL
    'input_RES2_capacity'       : 8287,         # base off-shore wind               # KF25 + FL
    'input_keol_reg'            : 234570000,                                        # FOA IDA
}
shock_case_params = {
    'input_nuclear_cap'         : 1000,         # shock nuclear capacity            # FL + LL
    'input_RES2_capacity'       : 6287,         # shock off-shore wind              # KF25 + FL
    'input_keol_reg'            : 923457000,                                        # (NOR_1GW: 234597000) (FOA: 923450000)
    'input_cshp_th_gr3'         : 577/100       # nuclear heat capacity             # FOA
}

### 1.2 Build outputfile

In [28]:
ref_path = r'..\ZipEnergyPLAN163\energyPlan Data\Data\IDA2045_nuclear_flexible.txt'

params = build_params(
    case=case,
    base_params=base_params,
    base_case_params=base_case_params,
    shock_case_params=shock_case_params,
)

lines, value_idx = load_energyplan_file(ref_path)

for name, new_val in params.items():
    if name not in value_idx:
        raise KeyError(f"Parameter name not found in file: {name!r}")
    idx = value_idx[name]
    lines[idx] = format_value(new_val)

with open(out_path, "w", encoding="utf-16") as f:
    f.writelines(lines)